In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os


import matplotlib.pyplot as plt
import seaborn as sns


DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SEED = 42


def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [4]:
#### 노트북 셀 가로 넓게 보기
from IPython.core.display import display, HTML

notebook_config = ""
notebook_config += "<style>"
notebook_config += "  .container { width:90% !important; }"
notebook_config += "</style>"
HTML(notebook_config)

In [7]:
# SEED 및 프로그램 버전 설정
IS_COLAB = True
PROGRAM_VERSION = "13"

In [8]:
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install bayesian-optimization


In [16]:
!pip install scikit-optimize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.2 MB/s eta 0:00:00


In [360]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")
import shutil
import time
from scipy.stats import gmean
from itertools import combinations
from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import TimeDistributed, LayerNormalization
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import PassiveAggressiveRegressor


from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

# # AdaBoost
from sklearn.ensemble import AdaBoostRegressor

# XGBoost
from xgboost import plot_importance
from xgboost import XGBRegressor

# lightgbm
from lightgbm import LGBMRegressor

# # catboost
from catboost import CatBoostRegressor

# metric
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV


import keras

print(tf.__version__)
print(keras.__version__)


2.13.0
2.13.1


In [20]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 32.2 MB/s eta 0:00:00


In [22]:
!pip install mordred


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.4 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=e70f7a680bb0c757e7b8913ab80bab81e0aa8173862311c20828d2d1897648b5
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [23]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.ipython_useSVG = True
# IPythonConsole.molSize = 500, 500

In [26]:
# 일단 colab 에서 실행하는 것으로 설정
DATA_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SUBMISSION_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
CLEANED_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
MODEL_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'

In [29]:
train_original = pd.read_csv(f'{DATA_DIR}train.csv')
# dev = pd.read_csv(f'{DATA_DIR}dev.csv')
test = pd.read_csv(f'{DATA_DIR}test.csv')
submission = pd.read_csv(f'{DATA_DIR}sample_submission.csv')

train = train_original # pd.concat([train_original, dev], axis=0)

train_original.shape, train.shape, test.shape

((3498, 11), (3498, 11), (483, 9))

In [30]:
calc = Calculator(descriptors, ignore_3D=False)
len(calc.descriptors)

1826

In [31]:
x_train = calc.pandas([Chem.MolFromSmiles(x) for x in train.SMILES])
x_train

100%|██████████| 3498/3498 [14:41<00:00,  3.97it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,21.379612,17.449011,0,0,35.689316,2.421199,4.745523,35.689316,1.274618,4.249154,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,16.539255,14.049653,0,0,26.575899,2.426398,4.757199,26.575899,1.265519,3.984419,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,17.475469,13.660693,2,1,29.802128,2.510668,4.982923,29.802128,1.354642,4.049690,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,27.857311,20.034364,0,1,45.884166,2.532483,4.973440,45.884166,1.310976,4.500758,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,15.722758,12.817176,0,0,26.308663,2.452930,4.905860,26.308663,1.315433,3.935574,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,19.701948,17.330258,0,0,30.902711,2.515832,4.792938,30.902711,1.236108,4.158311,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,21.148902,17.245220,0,0,35.887372,2.494190,4.985907,35.887372,1.329162,4.237338,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,14.348985,12.232287,0,0,23.546531,2.450594,4.723643,23.546531,1.239291,3.855798,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,14.458374,12.578196,0,0,23.936088,2.314213,4.623104,23.936088,1.259794,3.855415,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [32]:
train_complete = pd.concat([train, x_train], axis=1)
train_complete

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [33]:
x_test = calc.pandas([Chem.MolFromSmiles(x) for x in test.SMILES])
x_test

100%|██████████| 483/483 [02:01<00:00,  3.99it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,19.229782,15.428567,0,1,32.037339,2.392367,4.778964,32.037339,1.281494,4.137716,...,9.935713,72.422574,361.193631,6.946031,1607,34,126.0,144.0,7.527778,5.638889
1,21.229782,16.881776,0,0,35.824667,2.462328,4.912724,35.824667,1.326840,4.240550,...,10.330421,76.111616,370.152872,7.554140,1853,45,146.0,175.0,7.840278,5.972222
2,20.155669,15.481463,0,3,32.761443,2.378577,4.757154,32.761443,1.260056,4.175318,...,10.008163,60.614034,347.174610,7.386694,1905,37,132.0,150.0,8.388889,5.722222
3,21.079953,16.076404,0,0,35.577061,2.496055,4.855755,35.577061,1.368349,4.227377,...,10.288444,77.464779,345.133808,8.417898,1673,42,146.0,177.0,6.000000,5.583333
4,20.620135,16.065299,0,1,34.235212,2.546495,4.990622,34.235212,1.316739,4.210465,...,10.380312,75.874629,353.185175,7.207861,1656,47,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,16.291966,14.728760,0,0,28.248662,2.381151,4.715751,28.248662,1.284030,3.995519,...,9.862509,69.656439,306.230728,5.889052,1138,32,108.0,125.0,7.756944,5.222222
479,18.777305,15.822051,0,1,29.983837,2.559820,5.003321,29.983837,1.249327,4.116090,...,10.288852,73.647870,335.184506,6.840500,1280,42,130.0,157.0,9.201389,5.180556
480,20.648645,16.775304,0,0,33.716988,2.612365,5.113784,33.716988,1.296807,4.210713,...,10.447235,76.277596,349.142641,7.758725,1469,48,144.0,177.0,8.722222,5.611111
481,15.343821,13.245018,0,0,24.141151,2.379239,4.758478,24.141151,1.207058,3.909645,...,9.874728,53.565348,339.985883,11.332863,826,31,102.0,118.0,8.388889,4.361111


In [34]:
test_complete = pd.concat([test, x_test], axis=1)
test_complete

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,19.229782,...,9.935713,72.422574,361.193631,6.946031,1607,34,126.0,144.0,7.527778,5.638889
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,21.229782,...,10.330421,76.111616,370.152872,7.554140,1853,45,146.0,175.0,7.840278,5.972222
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,20.155669,...,10.008163,60.614034,347.174610,7.386694,1905,37,132.0,150.0,8.388889,5.722222
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,21.079953,...,10.288444,77.464779,345.133808,8.417898,1673,42,146.0,177.0,6.000000,5.583333
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,20.620135,...,10.380312,75.874629,353.185175,7.207861,1656,47,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13,16.291966,...,9.862509,69.656439,306.230728,5.889052,1138,32,108.0,125.0,7.756944,5.222222
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16,18.777305,...,10.288852,73.647870,335.184506,6.840500,1280,42,130.0,157.0,9.201389,5.180556
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72,20.648645,...,10.447235,76.277596,349.142641,7.758725,1469,48,144.0,177.0,8.722222,5.611111
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64,15.343821,...,9.874728,53.565348,339.985883,11.332863,826,31,102.0,118.0,8.388889,4.361111


In [35]:
# train_complete.to_csv(f"{CLEANED_DIR}train_mordred_1835.csv", index=False)
# test_complete.to_csv(f"{CLEANED_DIR}test_mordred_1835.csv", index=False)

In [312]:
train = pd.read_csv(f'{CLEANED_DIR}train_mordred_1835.csv')
test = pd.read_csv(f'{CLEANED_DIR}test_mordred_1835.csv')


### -> 이상치 제거 (총 18개)

In [313]:
# 1. 극단치만 판단해서 18개 제거
outliers = train[(train['MLM'] > 100.0) | (train['HLM'] > 100.0) | (train['AlogP'] < -3) | (train['Molecular_Weight'] > 800)
     | (train['Num_H_Acceptors'] > 14) | (train['Num_H_Donors'] > 9) | (train['Num_RotatableBonds'] > 20)
     | (train['LogD'] < -4) | (train['Molecular_PolarSurfaceArea'] > 250)]


In [314]:
outliers.index

Int64Index([ 179,  662,  834,  983, 1092, 1172, 1239, 1584, 2159, 2258, 2367,
            2410, 2586, 2711, 2948, 3157, 3247, 3403],
           dtype='int64')

In [315]:
len(outliers.index)

18

In [316]:
train = train.drop(index=outliers.index, axis=0)


In [317]:
train.shape # 3498 -> 3480

(3480, 1837)

In [318]:
duplicates = train[train.duplicated(subset=['SMILES'], keep=False)]

duplicates.sort_values(by='SMILES')


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
2276,TRAIN_2276,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,1.535,31.453,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
451,TRAIN_0451,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.310,24.670,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
2891,TRAIN_2891,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,55.950,69.950,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
543,TRAIN_0543,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,68.485,85.872,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
837,TRAIN_0837,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,63.522,62.488,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
366,TRAIN_0366,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,73.740,66.850,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
1085,TRAIN_1085,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,23.630,71.470,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
2848,TRAIN_2848,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,40.657,99.900,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
2096,TRAIN_2096,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,1.880,61.280,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000
1666,TRAIN_1666,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,5.494,25.397,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000


In [319]:
duplicates.shape # 이상치 제거로 27 -> 26쌍

(52, 1837)

In [320]:
processed_duplicate = duplicates.groupby('SMILES')['MLM','HLM'].mean().reset_index()
processed_duplicate

,SMILES,MLM,HLM
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615
1,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110
2,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690
3,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850
4,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385
5,CC(C)NC(=O)c1c(Cl)nn(C)c1NC(=O)c1cc(Br)nn1-c1n...,62.1085,68.1015
6,CC1CC(=O)N(c2ccc(-c3cccc(C#N)c3)cc2)N=C1c1ccc(...,43.1700,31.1300
7,CCCCC/N=c1\n(C)c(=O)nc2sccn12,2.3395,36.8145
8,CCOC(=O)CC1(NC(=O)N2Cc3c(sc4c3CCCC4)-n3cccc3C2...,3.4420,3.6015
9,CCc1nc2cc(Br)c(C(=O)OC)nc2n1CC(=O)c1ccccc1,73.5450,1.1345


In [321]:
tmp = pd.merge(processed_duplicate,duplicates,how='left',on='SMILES')
tmp

,SMILES,MLM_x,HLM_x,id,MLM_y,HLM_y,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_0451,0.310,24.670,3.556,262.309,3,0,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
1,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_2276,1.535,31.453,3.556,262.309,3,0,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
2,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_0543,68.485,85.872,2.172,337.372,4,2,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
3,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_2891,55.950,69.950,2.172,337.372,4,2,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
4,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0366,73.740,66.850,2.293,367.428,5,2,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
5,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0837,63.522,62.488,2.293,367.428,5,2,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
6,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_1085,23.630,71.470,1.684,381.450,7,1,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
7,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_2848,40.657,99.900,1.684,381.450,7,1,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
8,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_1666,5.494,25.397,2.843,360.490,3,0,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000
9,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_2096,1.880,61.280,2.843,360.490,3,0,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000


### -> 중복치 제거 (총 26개)

In [322]:
tmp = tmp.iloc[0::2]  # 첫 번째 행부터 시작하여 2씩 증가하여 행 선택
tmp.drop(['MLM_y','HLM_y'],axis =1,inplace=True)

In [323]:
tmp

,SMILES,MLM_x,HLM_x,id,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_0451,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
2,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_0543,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
4,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0366,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
6,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_1085,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
8,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_1666,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000
10,CC(C)NC(=O)c1c(Cl)nn(C)c1NC(=O)c1cc(Br)nn1-c1n...,62.1085,68.1015,TRAIN_0550,3.750,501.165,5,2,5,3.750,...,10.279970,80.683787,498.992590,11.088724,2216,46,152.0,180.0,11.583333,6.305556
12,CC1CC(=O)N(c2ccc(-c3cccc(C#N)c3)cc2)N=C1c1ccc(...,43.1700,31.1300,TRAIN_0328,4.449,381.427,4,1,3,4.446,...,10.313443,64.672912,381.147727,7.940578,2426,48,154.0,182.0,8.861111,6.388889
14,CCCCC/N=c1\n(C)c(=O)nc2sccn12,2.3395,36.8145,TRAIN_0859,2.484,252.336,4,0,4,2.474,...,9.659312,63.121099,252.104482,7.639530,538,24,84.0,98.0,5.805556,3.944444
16,CCOC(=O)CC1(NC(=O)N2Cc3c(sc4c3CCCC4)-n3cccc3C2...,3.4420,3.6015,TRAIN_0565,6.727,537.736,3,1,6,6.727,...,10.675330,91.336427,537.211984,7.461278,3860,66,206.0,251.0,9.784722,8.083333
18,CCc1nc2cc(Br)c(C(=O)OC)nc2n1CC(=O)c1ccccc1,73.5450,1.1345,TRAIN_0250,3.815,402.242,5,0,6,3.819,...,10.150152,74.109537,401.037503,9.781403,1475,41,130.0,155.0,8.750000,5.666667


In [324]:
# tmp.columns = ['SMILES', 'MLM', 'HLM','id',  'AlogP', 'Molecular_Weight',
#        'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
#        'Molecular_PolarSurfaceArea']

In [325]:
# processed_duplicate = tmp[['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
#        'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
#        'Molecular_PolarSurfaceArea']]

In [326]:
train = train.drop_duplicates(subset=['SMILES'], keep=False) #기존 train에서 중복치들을 우선 모두 제거
train.shape

(3428, 1837)

In [327]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [328]:
tmp.shape

(26, 1837)

In [329]:
tmp.rename(columns={'HLM_x': 'HLM', 'MLM_x': 'MLM'}, inplace=True)

In [330]:
train = pd.concat([train, tmp], axis=0, ignore_index=True)
train.shape

(3454, 1837)

In [331]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,...,10.344674,77.430819,340.143645,8.098658,1730,43,148.0,180.0,6.611111,5.500000
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222


# 피처 추가

In [332]:
train.shape, test.shape

((3454, 1837), (483, 1835))

In [333]:
test.columns

Index(['id', 'SMILES', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'ABC',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1835)

In [334]:
train.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1837)

In [336]:
# cols = ['mol','mode_label_proba']

# train = train.drop(columns = cols,axis=1)
# test = test.drop(columns = cols,axis=1)

In [337]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [338]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [339]:
from sklearn.metrics import make_scorer

rmse_score = make_scorer(rmse, greater_is_better=False)

In [340]:
# 결측치 채우기
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

In [341]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

target_1 = train['MLM']
target_2 = train['HLM']

In [342]:
for column in train_prop.columns:
    dtype = train_prop[column].dtype
    print(f"Column: {column}, Data Type: {dtype}")

Column: AlogP, Data Type: float64
Column: Molecular_Weight, Data Type: float64
Column: Num_H_Acceptors, Data Type: int64
Column: Num_H_Donors, Data Type: int64
Column: Num_RotatableBonds, Data Type: int64
Column: LogD, Data Type: float64
Column: Molecular_PolarSurfaceArea, Data Type: float64
Column: ABC, Data Type: float64
Column: ABCGG, Data Type: float64
Column: nAcid, Data Type: int64
Column: nBase, Data Type: int64
Column: SpAbs_A, Data Type: float64
Column: SpMax_A, Data Type: float64
Column: SpDiam_A, Data Type: float64
Column: SpAD_A, Data Type: float64
Column: SpMAD_A, Data Type: float64
Column: LogEE_A, Data Type: float64
Column: VE1_A, Data Type: float64
Column: VE2_A, Data Type: float64
Column: VE3_A, Data Type: float64
Column: VR1_A, Data Type: float64
Column: VR2_A, Data Type: float64
Column: VR3_A, Data Type: float64
Column: nAromAtom, Data Type: int64
Column: nAromBond, Data Type: int64
Column: nAtom, Data Type: int64
Column: nHeavyAtom, Data Type: int64
Column: nSpiro, 

In [343]:
object_columns_train = train_prop.select_dtypes(include=['object']).columns.tolist()
print(object_columns_train)

['AATS6dv', 'AATS7dv', 'AATS8dv', 'AATS6d', 'AATS7d', 'AATS8d', 'AATS6s', 'AATS7s', 'AATS8s', 'AATS6Z', 'AATS7Z', 'AATS8Z', 'AATS6m', 'AATS7m', 'AATS8m', 'AATS6v', 'AATS7v', 'AATS8v', 'AATS6se', 'AATS7se', 'AATS8se', 'AATS6pe', 'AATS7pe', 'AATS8pe', 'AATS6are', 'AATS7are', 'AATS8are', 'AATS6p', 'AATS7p', 'AATS8p', 'AATS6i', 'AATS7i', 'AATS8i', 'ATSC0c', 'ATSC1c', 'ATSC2c', 'ATSC3c', 'ATSC4c', 'ATSC5c', 'ATSC6c', 'ATSC7c', 'ATSC8c', 'AATSC0c', 'AATSC1c', 'AATSC2c', 'AATSC3c', 'AATSC4c', 'AATSC5c', 'AATSC6c', 'AATSC7c', 'AATSC8c', 'AATSC6dv', 'AATSC7dv', 'AATSC8dv', 'AATSC6d', 'AATSC7d', 'AATSC8d', 'AATSC6s', 'AATSC7s', 'AATSC8s', 'AATSC6Z', 'AATSC7Z', 'AATSC8Z', 'AATSC6m', 'AATSC7m', 'AATSC8m', 'AATSC6v', 'AATSC7v', 'AATSC8v', 'AATSC6se', 'AATSC7se', 'AATSC8se', 'AATSC6pe', 'AATSC7pe', 'AATSC8pe', 'AATSC6are', 'AATSC7are', 'AATSC8are', 'AATSC6p', 'AATSC7p', 'AATSC8p', 'AATSC6i', 'AATSC7i', 'AATSC8i', 'MATS1c', 'MATS2c', 'MATS3c', 'MATS4c', 'MATS5c', 'MATS6c', 'MATS7c', 'MATS8c', 'MATS6d

In [344]:
object_columns_test = test_prop.select_dtypes(include=['object']).columns.tolist()
print(object_columns_test)

['PNSA1', 'PNSA2', 'PNSA3', 'PNSA4', 'PNSA5', 'PPSA1', 'PPSA2', 'PPSA3', 'PPSA4', 'PPSA5', 'DPSA1', 'DPSA2', 'DPSA3', 'DPSA4', 'DPSA5', 'FNSA1', 'FNSA2', 'FNSA3', 'FNSA4', 'FNSA5', 'FPSA1', 'FPSA2', 'FPSA3', 'FPSA4', 'FPSA5', 'WNSA1', 'WNSA2', 'WNSA3', 'WNSA4', 'WNSA5', 'WPSA1', 'WPSA2', 'WPSA3', 'WPSA4', 'WPSA5', 'RNCS', 'RPCS', 'TASA', 'TPSA', 'RASA', 'RPSA', 'SpAbs_Dt', 'SpMax_Dt', 'SpDiam_Dt', 'SpAD_Dt', 'SpMAD_Dt', 'LogEE_Dt', 'SM1_Dt', 'VE1_Dt', 'VE2_Dt', 'VE3_Dt', 'VR1_Dt', 'VR2_Dt', 'VR3_Dt', 'DetourIndex', 'MAXsLi', 'MAXssBe', 'MAXssssBe', 'MAXssBH', 'MAXsssB', 'MAXssssB', 'MAXsCH3', 'MAXdCH2', 'MAXssCH2', 'MAXtCH', 'MAXdsCH', 'MAXaaCH', 'MAXsssCH', 'MAXddC', 'MAXtsC', 'MAXdssC', 'MAXaasC', 'MAXaaaC', 'MAXssssC', 'MAXsNH3', 'MAXsNH2', 'MAXssNH2', 'MAXdNH', 'MAXssNH', 'MAXaaNH', 'MAXtN', 'MAXsssNH', 'MAXdsN', 'MAXaaN', 'MAXsssN', 'MAXddsN', 'MAXaasN', 'MAXssssN', 'MAXsOH', 'MAXdO', 'MAXssO', 'MAXaaO', 'MAXsF', 'MAXsSiH3', 'MAXssSiH2', 'MAXsssSiH', 'MAXssssSi', 'MAXsPH2', 'MAXss

In [345]:
len(object_columns_train), len(object_columns_test)

(575, 403)

In [346]:
object_cols_total = object_columns_train + object_columns_test
object_cols_total = set(object_cols_total)
object_cols_total = list(object_cols_total)
len(object_cols_total)

575

In [347]:
train_prop.drop(columns =object_cols_total, axis=1, inplace=True )
train_prop.shape

(3454, 1258)

In [348]:
test_prop.drop(columns =object_cols_total, axis=1, inplace=True )
test_prop.shape

(483, 1258)

- 스케일링

In [349]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일링을 위한 객체 생성
scaler = MinMaxScaler()

# Train 데이터를 스케일링하고 변환
train_prop = scaler.fit_transform(train_prop)

# Test 데이터를 스케일링하고 변환
test_prop = scaler.transform(test_prop)


In [353]:
train_prop = pd.DataFrame(train_prop)
train_prop

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.488883,0.408759,0.416667,0.250,0.470588,0.525975,0.546835,0.405597,0.417590,0.000000,...,0.544667,0.579584,0.408596,0.137182,0.214867,0.305263,0.373016,0.345794,0.345075,0.466563
1,0.394950,0.231359,0.166667,0.125,0.117647,0.439098,0.336496,0.264882,0.279238,0.000000,...,0.485644,0.436791,0.231203,0.127029,0.072064,0.252632,0.253968,0.242991,0.222861,0.261275
2,0.345312,0.224110,0.416667,0.000,0.176471,0.392184,0.283695,0.292099,0.263407,0.666667,...,0.566000,0.451774,0.224119,0.110023,0.087006,0.263158,0.285714,0.283489,0.126528,0.301711
3,0.619183,0.577331,0.500000,0.000,0.294118,0.543238,0.428154,0.593912,0.522812,0.000000,...,0.725217,0.690095,0.577135,0.104858,0.384150,0.526316,0.571429,0.551402,0.411574,0.594090
4,0.409255,0.172104,0.250000,0.000,0.058824,0.452286,0.187773,0.241146,0.229077,0.000000,...,0.509591,0.209825,0.172085,0.122153,0.061850,0.221053,0.230159,0.221184,0.178289,0.242613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,0.605222,0.301135,0.333333,0.125,0.117647,0.644022,0.328255,0.400066,0.353467,0.000000,...,0.633958,0.559821,0.301096,0.161673,0.153395,0.336842,0.396825,0.392523,0.195543,0.370140
3450,0.577042,0.682652,0.416667,0.000,0.411765,0.585918,0.300608,0.657112,0.625926,0.000000,...,0.718524,0.719286,0.680891,0.162977,0.450728,0.547368,0.619048,0.585670,0.477354,0.674961
3451,0.578938,0.514433,0.500000,0.125,0.235294,0.609495,0.528437,0.554507,0.550021,0.000000,...,0.746847,0.660777,0.514244,0.186196,0.262909,0.526316,0.547619,0.538941,0.417326,0.531882
3452,0.110393,0.381764,0.583333,0.000,0.235294,0.174952,0.505055,0.421798,0.415727,0.000000,...,0.577309,0.584540,0.381759,0.128507,0.203707,0.326316,0.396825,0.376947,0.267434,0.451011


In [354]:
test_prop = pd.DataFrame(test_prop)
test_prop

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.435626,0.338954,0.333333,0.250,0.411765,0.476103,0.428921,0.343099,0.335359,0.0,...,0.495111,0.485416,0.338802,0.091685,0.141763,0.242105,0.309524,0.280374,0.242991,0.385692
1,0.258445,0.354877,0.416667,0.000,0.176471,0.312260,0.311772,0.401242,0.394503,0.0,...,0.629119,0.540223,0.354851,0.128609,0.165027,0.357895,0.388889,0.376947,0.259166,0.423017
2,0.576525,0.313726,0.333333,0.500,0.294118,0.608376,0.429400,0.370016,0.337512,0.0,...,0.519708,0.309983,0.313691,0.118442,0.169945,0.273684,0.333333,0.299065,0.287563,0.395023
3,0.362720,0.310045,0.416667,0.000,0.117647,0.408967,0.373581,0.396886,0.361725,0.0,...,0.614867,0.560326,0.310035,0.181057,0.148005,0.326316,0.388889,0.383178,0.163911,0.379471
4,0.313082,0.324475,0.333333,0.000,0.117647,0.279012,0.277466,0.383519,0.361273,0.0,...,0.646057,0.536702,0.324457,0.107583,0.146397,0.378947,0.380952,0.380062,0.304817,0.385692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0.570579,0.240375,0.166667,0.125,0.411765,0.601742,0.248622,0.257693,0.306877,0.0,...,0.470257,0.444321,0.240349,0.027504,0.097409,0.221053,0.238095,0.221184,0.254853,0.339036
479,0.155636,0.292214,0.416667,0.000,0.058824,0.126758,0.320636,0.329945,0.351373,0.0,...,0.615006,0.503620,0.292213,0.085277,0.110838,0.326316,0.325397,0.320872,0.329619,0.334370
480,0.362461,0.317251,0.250000,0.125,0.176471,0.408728,0.318528,0.384347,0.390170,0.0,...,0.668779,0.542689,0.317216,0.141032,0.128712,0.389474,0.380952,0.383178,0.304817,0.382582
481,0.276112,0.302479,0.250000,0.250,0.117647,0.299313,0.318145,0.230130,0.246490,0.0,...,0.474406,0.205264,0.300814,0.358055,0.067902,0.210526,0.214286,0.199377,0.287563,0.242613


#### CatBoost

In [356]:
!pip install CatBoost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.3 MB/s eta 0:00:00


In [362]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop, target_1)):
    # 학습데이터
    x_train = train_prop.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [29:41, 356.22s/it]

[30.51499694297415, 32.45549577695499, 31.71205864542828, 30.218236226795362, 31.003506913236507]


31.180858901077862

In [363]:
model.fit(train_prop, target_1)
cat_pred_1 = model.predict(test_prop)

model.fit(train_prop, target_2)
cat_pred_2 = model.predict(test_prop)

In [ ]:
# import joblib

# # 모델 저장
# joblib.dump(mlm_model, 'mlm_model.pkl')
# joblib.dump(hlm_model, 'hlm_model.pkl')

#### AdaBoost

In [367]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop, target_1)):
    # 학습데이터
    x_train = train_prop.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = AdaBoostRegressor(random_state=SEED)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = AdaBoostRegressor(random_state=SEED)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [02:54, 34.82s/it]

[32.36238469218258, 33.36197027802679, 33.24936591196395, 32.46851128996565, 32.980572138938186]


32.88456086221543

In [368]:
model.fit(train_prop, target_1)
ada_pred_1 = model.predict(test_prop)

model.fit(train_prop, target_2)
ada_pred_2 = model.predict(test_prop)

#### randomforest

In [370]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop, target_1)):
    # 학습데이터
    x_train = train_prop.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = RandomForestRegressor(random_state=SEED)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = RandomForestRegressor(random_state=SEED)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [31:25, 377.08s/it]

[30.978366949192015, 32.575945789114286, 31.787306579152258, 30.688974175121523, 31.32539794849395]


31.47119828821481

In [371]:
model.fit(train_prop, target_1)
rf_pred_1 = model.predict(test_prop)

model.fit(train_prop, target_2)
rf_pred_2 = model.predict(test_prop)

#### XGBRegressor

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop, target_1)):
    # 학습데이터
    x_train = train_prop.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = XGBRegressor(random_state=SEED)
    model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric='rmse')
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    # HLM
    model = XGBRegressor(random_state=SEED)
    model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric='rmse')
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

In [ ]:
model.fit(train_prop, target_1)
xgb_pred_1 = model.predict(test_prop)

model.fit(train_prop, target_2)
xgb_pred_2 = model.predict(test_prop)

#### LGBMRegressor

In [365]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop, target_1)):
    # 학습데이터
    x_train = train_prop.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = LGBMRegressor(random_state=SEED, objective='regression')
    model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric="rmse")
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    # HLM
    model = LGBMRegressor(random_state=SEED, objective='regression')
    model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric="rmse")
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)


    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

0it [00:00, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055501 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227331
[LightGBM] [Info] Number of data points in the train set: 2763, number of used features: 1078
[LightGBM] [Info] Start training from score 37.549743
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034038 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227331
[LightGBM] [Info] Number of data points in the train set: 2763, number of used features: 1078
[LightGBM] [Info] Start training from score 53.023585


1it [00:15, 15.17s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032010 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227189
[LightGBM] [Info] Number of data points in the train set: 2763, number of used features: 1075
[LightGBM] [Info] Start training from score 37.414670
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034451 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227189
[LightGBM] [Info] Number of data points in the train set: 2763, number of used features: 1075
[LightGBM] [Info] Start training from score 53.194288


2it [00:29, 14.72s/it]

...

4it [00:58, 14.40s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032311 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227317
[LightGBM] [Info] Number of data points in the train set: 2764, number of used features: 1080
[LightGBM] [Info] Start training from score 37.282076
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047834 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227317
[LightGBM] [Info] Number of data points in the train set: 2764, number of used features: 1080
[LightGBM] [Info] Start training from score 52.875017


5it [01:12, 14.44s/it]

[30.9661831015655, 32.58627700510118, 32.070671824592225, 30.33145646821403, 31.434068956172148]


31.477731471129015

In [366]:
model.fit(train_prop, target_1)
lgbm_pred_1 = model.predict(test_prop)

model.fit(train_prop, target_2)
lgbm_pred_2 = model.predict(test_prop)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 228302
[LightGBM] [Info] Number of data points in the train set: 3454, number of used features: 1081
[LightGBM] [Info] Start training from score 37.276169
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041263 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 228302
[LightGBM] [Info] Number of data points in the train set: 3454, number of used features: 1081
[LightGBM] [Info] Start training from score 53.008790


In [ ]:
# mlm_pred = (cat_pred_1 + ada_pred_1 + rf_pred_1 + xgb_pred_1 + lgbm_pred_1) / 5
# hlm_pred = (cat_pred_2 + ada_pred_2 +rf_pred_2 + xgb_pred_2 + lgbm_pred_2) / 5

In [372]:
mlm_pred = (cat_pred_1 + rf_pred_1 + lgbm_pred_1) / 3
hlm_pred = (cat_pred_2 + rf_pred_2 + lgbm_pred_2) / 3

In [373]:
submission['MLM'] = mlm_pred
submission['HLM'] = hlm_pred

In [374]:
submission

,id,MLM,HLM
0,TEST_000,23.685651,47.797643
1,TEST_001,56.479102,77.945344
2,TEST_002,39.223625,53.439247
3,TEST_003,45.507721,73.584156
4,TEST_004,66.290823,78.858035
...,...,...,...
478,TEST_478,17.544773,36.862662
479,TEST_479,75.631404,88.271733
480,TEST_480,54.898030,72.452000
481,TEST_481,63.882227,69.706789


In [375]:
submission.to_csv("submission_ML17(mordred3).csv", index=False)

In [376]:
submission

,id,MLM,HLM
0,TEST_000,23.685651,47.797643
1,TEST_001,56.479102,77.945344
2,TEST_002,39.223625,53.439247
3,TEST_003,45.507721,73.584156
4,TEST_004,66.290823,78.858035
...,...,...,...
478,TEST_478,17.544773,36.862662
479,TEST_479,75.631404,88.271733
480,TEST_480,54.898030,72.452000
481,TEST_481,63.882227,69.706789


In [377]:
sub_16 = pd.read_csv('submission_ML16(preprocess).csv')

In [378]:
sub_16

,id,MLM,HLM
0,TEST_000,34.270580,55.729152
1,TEST_001,60.907718,63.722749
2,TEST_002,30.194104,51.532413
3,TEST_003,57.818966,74.373008
4,TEST_004,63.833841,80.345176
...,...,...,...
478,TEST_478,7.256792,19.426390
479,TEST_479,78.237783,86.651548
480,TEST_480,48.114328,86.437328
481,TEST_481,59.538181,60.353697


In [379]:
sub_10 = pd.read_csv('submission_ML10(preprocess).csv')

In [380]:
sub_10

,id,MLM,HLM
0,TEST_000,31.333722,50.995665
1,TEST_001,61.914548,69.861081
2,TEST_002,20.535638,56.854744
3,TEST_003,60.869370,75.067627
4,TEST_004,65.219084,75.347482
...,...,...,...
478,TEST_478,2.029218,25.178712
479,TEST_479,82.603555,87.825730
480,TEST_480,42.525753,69.285844
481,TEST_481,51.875330,60.435923
